In [1]:
import json

In [2]:
import time

In [3]:
from tqdm import tqdm

In [4]:
import re
import ast

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, LogitsProcessor, LogitsProcessorList
from functools import reduce

/data/wang/zhenyub/condaenvs/cenv_x86/lib/python3.8/site-packages/transformers/utils/hub.py:127: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [6]:
import transformers
access_token = "KEY"

In [7]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Meta-Llama-3.1-8B-Instruct", token=access_token)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Meta-Llama-3.1-8B-Instruct", torch_dtype=torch.float16, low_cpu_mem_usage=True, device_map="auto", token=access_token
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [8]:
import random

In [13]:
rand_list_8 = []
rand_list_8_sorted = []
n = 8
m = 10
for j in range(m):
    rand_list = []
    rand_list_sorted = []
    for i in range(n):
        current = random.randint(0,9)
        rand_list.append(current)
    rand_list_8.append(rand_list)
    rand_list_8_sorted.append(rand_list.sort())

In [14]:
rand_list_12 = []
rand_list_12_sorted = []
n = 12
m = 10
for j in range(m):
    rand_list = []
    rand_list_sorted = []
    for i in range(n):
        current = random.randint(0,9)
        rand_list.append(current)
    rand_list_12.append(rand_list)
    rand_list_12_sorted.append(rand_list.sort())

In [16]:
rand_list_16 = []
rand_list_16_sorted = []
n = 16
m = 10
for j in range(m):
    rand_list = []
    rand_list_sorted = []
    for i in range(n):
        current = random.randint(0,9)
        rand_list.append(current)
    rand_list_16.append(rand_list)
    rand_list.sort()
    rand_list_16_sorted.append(rand_list)

In [35]:
rand_list.sort()

In [36]:
def gen_sorting_examples(n, m, rand_range):
    rand_list_n = []
    rand_list_sorted_n = []
    for j in range(m):
        rand_list = []
        for i in range(n):
            current = random.randint(rand_range[0],rand_range[1])
            rand_list.append(current)
        rand_list_n.append(rand_list)
        rand_list.sort()
        rand_list_sorted_n.append(rand_list)
    return rand_list_n, rand_list_sorted_n

In [19]:
def Answer_Agent_IO(input='', num_seq=1):
    message = \
    f"""<Instruction> Sort the following list of numbers in ascending order. Output only the sorted list of numbers, no additional text. </Instruction>

<Examples>
Input: [5, 1, 1, 2, 4, 8, 1, 9, 5, 1, 3, 3, 9, 7]
Output: [1, 1, 1, 1, 2, 3, 3, 4, 5, 5, 7, 8, 9, 9]

Input: [3, 7, 0, 2, 8, 1, 2, 2, 2, 4, 7, 8, 5, 5, 3, 9, 4, 3, 5, 6, 6, 4, 4, 5, 2, 0, 9, 3, 3, 9, 2, 1]
Output: [0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 9]

Input: [4, 4, 9, 7, 9, 7, 0, 0, 4, 9, 1, 7, 9, 5, 8, 7, 5, 6, 3, 8, 6, 7, 5, 8, 5, 0, 6, 3, 7, 0, 5, 3, 7, 5, 2, 4, 4, 9, 0, 7, 8, 2, 7, 7, 7, 2, 1, 3, 9, 9, 7, 9, 6, 6, 4, 5, 4, 2, 0, 8, 9, 0, 2, 2]
Output: [0, 0, 0, 0, 0, 0, 0, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 7, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9, 9]
</Examples>

Input: {input}"""
    
    messages = [{"role": "system", "content": \
                     "You are a faithful agent that answer questions accurately."},\
                {"role": "user", "content": message}]
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=256,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        num_return_sequences=num_seq,
    )

    responses = []
    n = 0
    while n < num_seq:
        responses.append(tokenizer.decode(outputs[n][input_ids.shape[-1]:], skip_special_tokens=True))
        n += 1
    
    return responses

In [44]:
rand_range = [0,9]
rand_list_8, rand_list_8_sorted = gen_sorting_examples(8, 100, rand_range)
agent_answer = []
for rl in rand_list_8:
    agent_answer.append(Answer_Agent_IO(str(rl), 1))

In [47]:
correct = 0
for aa, golden in zip(agent_answer, rand_list_8_sorted):
    if ast.literal_eval(aa[0]) == golden:
        correct += 1
correct

70

In [48]:
rand_range = [0,9]
rand_list_12, rand_list_12_sorted = gen_sorting_examples(12, 100, rand_range)
agent_answer_12 = []
for rl in rand_list_12:
    agent_answer_12.append(Answer_Agent_IO(str(rl), 1))

In [49]:
correct_12 = 0
for aa, golden in zip(agent_answer_12, rand_list_12_sorted):
    if ast.literal_eval(aa[0]) == golden:
        correct_12 += 1
correct_12

37

In [50]:
rand_range = [0,9]
rand_list_16, rand_list_16_sorted = gen_sorting_examples(16, 100, rand_range)
agent_answer_16 = []
for rl in rand_list_16:
    agent_answer_16.append(Answer_Agent_IO(str(rl), 1))

In [51]:
correct_16 = 0
for aa, golden in zip(agent_answer_16, rand_list_16_sorted):
    if ast.literal_eval(aa[0]) == golden:
        correct_16 += 1
correct_16

43

In [52]:
rand_range = [0,9]
rand_list_20, rand_list_20_sorted = gen_sorting_examples(20, 100, rand_range)
agent_answer_20 = []
for rl in rand_list_20:
    agent_answer_20.append(Answer_Agent_IO(str(rl), 1))

In [54]:
correct_20 = 0
for aa, golden in zip(agent_answer_20, rand_list_20_sorted):
    try:
        if ast.literal_eval(aa[0]) == golden:
            correct_20 += 1
    except:
        continue
correct_20

9

In [55]:
def Answer_Agent_CoT_3Shot(input, num_seq=1):
    message = \
    f"""<Instruction> Sort the following list of numbers in ascending order. You can generate any intermediate lists, but the final output should be the sorted list of numbers, prefixed with "Output: ". </Instruction>

<Approach>
To sort the list of numbers follow these steps:
1. Split the list of numbers into two to four unsorted sublists, each containing an equal number of elements from the original list (make sure they don't overlap).
2. Sort each of the unsorted sublists.
3. Merge the sorted sublists into a single sorted list using the merging algorithm from merge sort.
</Approach>

<Examples>
Input: [4, 5, 3, 3, 7, 3, 0, 5, 0, 2, 8, 0, 2, 1, 6, 9]
Unsorted Subarrays:
[4, 5, 3, 3, 7, 3, 0, 5]
[0, 2, 8, 0, 2, 1, 6, 9]
Sorted Subarrays:
[0, 3, 3, 3, 4, 5, 5, 7]
[0, 0, 1, 2, 2, 6, 8, 9]
Output: [0, 0, 0, 1, 2, 2, 3, 3, 3, 4, 5, 5, 6, 7, 8, 9]

Input: [6, 4, 5, 7, 5, 6, 9, 7, 6, 9, 4, 6, 9, 8, 1, 9, 2, 4, 9, 0, 7, 6, 5, 6, 6, 2, 8, 3, 9, 5, 6, 1]
Unsorted Subarrays:
[6, 4, 5, 7, 5, 6, 9, 7, 6, 9, 4, 6, 9, 8, 1, 9]
[2, 4, 9, 0, 7, 6, 5, 6, 6, 2, 8, 3, 9, 5, 6, 1]
Sorted Subarrays:
[1, 4, 4, 5, 5, 6, 6, 6, 6, 7, 7, 8, 9, 9, 9, 9]
[0, 1, 2, 2, 3, 4, 5, 5, 6, 6, 6, 6, 7, 8, 9, 9]
Output: [0, 1, 1, 2, 2, 3, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 8, 8, 9, 9, 9, 9, 9, 9]

Input: [3, 7, 0, 2, 8, 1, 2, 2, 2, 4, 7, 8, 5, 5, 3, 9, 4, 3, 5, 6, 6, 4, 4, 5, 2, 0, 9, 3, 3, 9, 2, 1, 9, 3, 1, 8, 1, 8, 6, 0, 1, 6, 1, 7, 4, 4, 6, 3, 3, 7, 9, 3, 6, 0, 3, 4, 5, 6, 6, 9, 9, 9, 7, 3]
Unsorted Subarrays:
[3, 7, 0, 2, 8, 1, 2, 2, 2, 4, 7, 8, 5, 5, 3, 9]
[4, 3, 5, 6, 6, 4, 4, 5, 2, 0, 9, 3, 3, 9, 2, 1]
[9, 3, 1, 8, 1, 8, 6, 0, 1, 6, 1, 7, 4, 4, 6, 3]
[3, 7, 9, 3, 6, 0, 3, 4, 5, 6, 6, 9, 9, 9, 7, 3]
Sorted Subarrays:
[0, 1, 2, 2, 2, 2, 3, 3, 4, 5, 5, 7, 7, 8, 8, 9]
[0, 1, 2, 2, 3, 3, 3, 4, 4, 4, 5, 5, 6, 6, 9, 9]
[0, 1, 1, 1, 1, 3, 3, 4, 4, 6, 6, 6, 7, 8, 8, 9]
[0, 3, 3, 3, 3, 4, 5, 6, 6, 6, 7, 7, 9, 9, 9, 9]
Output: [0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 5, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 6, 7, 7, 7, 7, 7, 8, 8, 8, 8, 9, 9, 9, 9, 9, 9, 9, 9]
</Examples>

Input: {input}"""
    
    messages = [{"role": "system", "content": \
                     "You are a faithful agent that answer questions accurately."},\
                {"role": "user", "content": message}]
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        num_return_sequences=num_seq,
    )

    responses = []
    n = 0
    while n < num_seq:
        responses.append(tokenizer.decode(outputs[n][input_ids.shape[-1]:], skip_special_tokens=True))
        n += 1
    
    return responses

In [56]:
rand_range = [0,9]
rand_list_8, rand_list_8_sorted = gen_sorting_examples(8, 100, rand_range)
agent_answer = []
for rl in rand_list_8:
    try:
        agent_answer.append(Answer_Agent_CoT_3Shot(str(rl), 1))
    except:
        agent_answer.append(['1'])

In [57]:
correct = 0
for aa, golden in zip(agent_answer, rand_list_8_sorted):
    if ast.literal_eval(aa[0]) == golden:
        correct += 1
correct

0

In [58]:
agent_answer

[['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1'],
 ['1']]

In [22]:
Answer_Agent_CoT_3Shot('[1, 5, 2, 6, 3, 7, 7, 4, 8, 2, 4, 3, 9, 6]', 3)

["To sort the list of numbers, we will follow the steps as described in the approach.\n\n1. Split the list of numbers into two unsorted sublists, each containing an equal number of elements from the original list (make sure they don't overlap).\n2. Sort each of the unsorted sublists.\n3. Merge the sorted sublists into a single sorted list using the merging algorithm from merge sort.\n\nHere are the steps for the given input:\n\nInput: [1, 5, 2, 6, 3, 7, 7, 4, 8, 2, 4, 3, 9, 6]\n\nSplitting the list into two sublists:\nSublist 1: [1, 5, 2, 6, 3, 7, 7, 4]\nSublist 2: [8, 2, 4, 3, 9, 6]\n\nSorting each sublist:\nSublist 1: [1, 2, 3, 4, 5, 6, 7, 7]\nSublist 2: [2, 3, 4, 6, 8, 9]\n\nMerging the sorted sublists:\nMerging [1, 2, 3, 4, 5, 6, 7, 7] and [2, 3, 4, 6, 8, 9] into a single sorted list:\n[1, 2, 2, 3, 3, 4, 4, 5, 6, 6, 7, 7, 8, 9]\n\nOutput: [1, 2, 2, 3, 3, 4, 4, 5, 6, 6, 7, 7, 8, 9]",
 "To sort the list of numbers, we will follow the steps outlined in the approach.\n\n1. Split the li

In [ ]:
agent_answer.split("Output: ")[1]

In [22]:
s = '###123###'
ast.literal_eval(re.search('###(.*)###', s).group(1))

123

In [24]:
ord('A')

65

In [33]:
all_answer_lm3_text_io = []
ori_answers_io = []
all_counts = -1
for s in tqdm(list(arc_challenge['validation'])[0:100]):
    all_counts += 1
    ori_question = s['question']

    choices = s['choices']['text']
    choices_text = ''
    i = 65
    for c in choices:
        choices_text += f'{chr(i)}) {c}; '
        i += 1

    response = Answer_Agent_IO(ori_question, choices_text, 1)[0]
    ori_answers_io.append(response)
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_io.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_io.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_io.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_io.append('')
                continue

100%|██████████| 100/100 [01:02<00:00,  1.61it/s]


In [35]:
all_answer_lm3_text_cot_0s = []
all_counts = -1

for s in tqdm(list(arc_challenge['validation'])[0:100]):
    all_counts += 1
    ori_question = s['question']

    choices = s['choices']['text']
    choices_text = ''
    i = 65
    for c in choices:
        choices_text += f'{chr(i)}) {c}; '
        i += 1

    response = Answer_Agent_CoT_0Shot(ori_question, choices_text, 1)[0]
    try:
        response_format = re.search('###(.*)###', response).group(1)
        try:
            answer_current = ast.literal_eval(response_format)
        except:
            answer_current = response_format
        all_answer_lm3_text_cot_0s.append(answer_current)
    except:
        try:
            answer_current = ast.literal_eval(response)
            all_answer_lm3_text_cot_0s.append(answer_current)
        except:
            try:
                response_format = response.split('Answer: ')[1]
                answer_current = ast.literal_eval(response_format)
                all_answer_lm3_text_cot_0s.append(answer_current)
            except:
                print('no answer provided')
                print(response)
                all_answer_lm3_text_cot_0s.append('')
                continue

  0%|          | 0/100 [00:00<?, ?it/s]

100%|██████████| 100/100 [00:44<00:00,  2.27it/s]


In [36]:
import sys
import re
import string
from collections import Counter
import pickle

In [32]:
def normalize_answer(s):

    def remove_articles(text):
        return re.sub(r'\b(a|an|the)\b', ' ', text)

    def white_space_fix(text):
        return ' '.join(text.split())

    def remove_punc(text):
        exclude = set(string.punctuation)
        return ''.join(ch for ch in text if ch not in exclude)

    def lower(text):
        return text.lower()

    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [36]:
with open('answers_lm3_io.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_io))

In [48]:
with open('answers_lm3_io_ori.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in ori_answers_io))

In [37]:
with open('answers_lm3_cot_0Shot.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_cot_0s))

In [40]:
with open('answers_lm3_cot_2Shot.txt', 'w') as f:
    f.write('\n'.join(str(ans) for ans in all_answer_lm3_text_cot))

In [38]:
with open('answers_golden.txt', 'w') as f:
    f.write('\n'.join(s['answerKey'] for s in list(arc_challenge['validation'])[0:100]))

In [ ]:
def Answer_Agent_debate(question='', prior=[], others='', num_seq=1):
    message = \
    f"""Given a question, and the reasoning processes from another agent, recheck your thoughts and answer. \
    Make sure to include your answer in the format: ###answer###. \
    Question: {question} \
    Reasoning from another agent: {others} \
    Thought: \
    """
    
    messages = [{"role": "system", "content": \
                        "You are a faithful agent that answer mathematical questions accurately."}]
    for p in prior:
        messages.append({"role": "user", "content": p})
    messages.append({"role": "user", "content": message})
    
    input_ids = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        return_tensors="pt"
    ).to(model.device)

    terminators = [
        tokenizer.eos_token_id,
        tokenizer.convert_tokens_to_ids("<|eot_id|>")
    ]

    outputs = model.generate(
        input_ids,
        max_new_tokens=512,
        eos_token_id=terminators,
        pad_token_id=tokenizer.eos_token_id,
        do_sample=True,
        temperature=0.5,
        top_p=0.9,
        num_return_sequences=num_seq,
    )

    responses = []
    n = 0
    while n < num_seq:
        responses.append(tokenizer.decode(outputs[n][input_ids.shape[-1]:], skip_special_tokens=True))
        n += 1
    
    return responses